# Predicting school STAAR scores

In [3]:
import pandas as pd
import prepare
import scipy.stats as stats

## Project Goals:

<div class="alert alert-block alert-info">
<b>Highlights:</b>
</br>

* identify features of teachers/students that have a relationship with STAAR score percentage.
</br>

* Visualize relationships between variables
</br>

* use regression models to predict likely STAAR score for each test (English 1, English 2, Algebra, US History, Biology.

## Acquire

<div class="alert alert-block alert-info">
<b>Highlights:</b>
</br>

* Data was obtained from the Texas Education Agency at tea.gov
</br>

* Data was obtained 3/13/2023
</br>

* Web scraping function was created to scrape features of primary interest and placed into one DataFrame
</br>

* Each row represents a studen and each column is a feature of that student
    

In [9]:
school = prepare.clean_df()

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
def tts(df, stratify=None):
    '''
    removing your test data from the data
    '''
    train_validate, test=train_test_split(df, 
                                 train_size=.8, 
                                 random_state=137,
                                 stratify=None)
    '''
    splitting the remaining data into the train and validate groups
    '''            
    train, validate =train_test_split(train_validate, 
                                      test_size=.3, 
                                      random_state=137,
                                      stratify=None)
    return train, validate, test

In [13]:
train, val, test = tts(school, school.extracurricular_expend)

In [16]:
def extra_subs(train):

    '''    
    This function executes pearson r test testing for correlation 
    between each subject's passing rate and student teacher ratio.
    '''

    subjects = ['english_1', 'english_2', 'algebra', 'biology', 'history']

    data = []
    for i in subjects:

        r, p = stats.pearsonr(x=train.extracurricular_expend, y=train[i])
        data.append({
                     'subject': i, 
                     'correlation': r, 
                     'p-value': p
                     })
        
    df = pd.DataFrame(data)
    
    return df

In [17]:
extra_subs(train)

,subject,correlation,p-value
0,english_1,0.147849,3.472362e-05
1,english_2,0.148891,3.051963e-05
2,algebra,0.185599,1.849293e-07
3,biology,0.166587,2.986320e-06
4,history,0.123032,5.833295e-04
